# TODO:

- Ignore NaN values when calculating moving averages
- Clean dfs that have column headers but don't have data
- Create mechanism that will cut dataframe into hourly, 12 hour, daily, etc slices


# Imports and Function Declarations

In [1]:
import urllib
import requests
import pickle
from pandas.io.json import json_normalize
import json
import pandas as pd
#pd.options.mode.chained_assignment = None
import datetime
import time
import pytz
import math
import numpy as np
from scipy.stats import zscore
import sys


def generate_item_lookup(file_name):
    """
        Reads in item name/ids from file
        
            @param file_name: file containing item name/ids
    """
    return pd.read_csv(file_name, skiprows=[])

def get_item_records_from_pickle(file_name):
    """
        This method loads data from the Moophan provided Pickle files instead of from 
        the deprecated RSBuddy API
        
            @param file_name: the name of the pickle file from which to load data
    """
    # Open file and read into data structure
    with open(file_name, 'rb') as row:
        ge_data = pickle.load(row)
        
    # Get all item name/ids for lookup
    item_lookup = generate_item_lookup('./item_key.csv')
        
    # Create data frame from loaded data by iterating through all contained items
    item_records = pd.DataFrame(columns=['id', 'name', 'data'])
    item_keys = list(ge_data.keys())
    curr = 0
    for key in item_keys:
        #Get name of item
        name = item_lookup.loc[item_lookup['id'] == int(key)]['name'].item()
        sys.stdout.write('\rProcessing Item: {0} / {1}'.format(curr, len(item_keys)))
        sys.stdout.flush()
        #Grab data
        data = pd.DataFrame(ge_data[key])
        item_records = item_records.append({'id':int(key), 'name': name, 'data':data}, ignore_index=True)
        curr += 1
        
    return item_records

def clean_initial_data(item_records):
    """
        Will perform prelim cleaning on dataset
            
            @param item_records: pandas dataframe representing dataset
    """
    # Find all indices to drop
    drop_indices = []
    for idx, row in item_records.iterrows():
        #Drop row if item data empty
        if(row['data'].empty):
            drop_indices.append(idx)
            
        #Drop row if data doesn't have all columns
        columns = set(['ts', 'buyingPrice', 'buyingCompleted', 'sellingPrice', 'sellingCompleted', 'overallPrice', 'overallCompleted'])
        if not columns.issubset(set(row['data'].columns)):
            drop_indices.append(idx)
              
    #Make drop_indices unique
    drop_indices = list(set(drop_indices)) 
    #Drop rows with empty data
    item_records = item_records.drop(drop_indices)    
    #Reset index
    item_records.reset_index(drop=True, inplace=True)
    
    print('Data Cleaning Complete')
    
    return item_records
    

def format_item_record_dates(item_records):
    """ 
        Converts timestamp to Unix seconds, tacks on formatted date field, and creates Unix 
        seconds from most recent datapoint, and Unix seconds position from Jan-1.
        
            @param item_records: the master item data structure
    """
    #Get most recent timestamp in all the data (highest value) 
    most_recent_ts = 0
    for _, row in item_records.iterrows():
        max_ts = row['data']['ts'].max()
        if max_ts > most_recent_ts:
            most_recent_ts = max_ts / 1000 #Convert to seconds
    
    #Get the timestamp of Jan-1 for 2015, 2016, 2017, and 2018
    jan1_timestamps = {}
    time_adjust = 18000 #Seconds ahead of EST
    for year in [2015, 2016, 2017, 2018]:
        jan1 = datetime.date(year, 1, 1)
        jan1_ts = time.mktime(jan1.timetuple())
        jan1_timestamps[str(year)] = jan1_ts - time_adjust
        
    total_items = item_records.shape[0]
    for idx, row in item_records.iterrows():
        #Convert timestamp from milliseconds to seconds
        row['data']['ts'] = row['data']['ts'] / 1000
        #Append column to indicate time delta from most recent record in dataset
        row['data']['tsFromCurrent'] = most_recent_ts - row['data']['ts']
        #Append column that converts ts to datetime object
        row['data']['date'] = pd.to_datetime(row['data']['ts'], unit='s')
        row['data']['year'] = row['data']['date'].dt.strftime('%Y')
        #Append a column representing the YTD seconds
        row['data']['tsYtd'] = row['data']['ts'] - pd.Series([jan1_timestamps[year] for year in row['data']['year']])
        row['data'].drop(columns=['year'], inplace=True)
        
        sys.stdout.write('\rProcessing Item: {0}/{1}'.format(idx, total_items))
        sys.stdout.flush()
        
    sys.stdout.write('\rDate Formatting Complete.')
    sys.stdout.flush()
            
    return item_records

def generate_train_and_test_deltas(item_records):    
    """
        Calculate percent difference from previous record for each relevant column. 
            
            @param item_records: the master item data structure
    """
    #Iterate through each item, and calculate percentages for their data
    for idx, row in item_records.iterrows():
        sys.stdout.write('\rProcessing Item: {0}/{1}'.format(idx, item_records.shape[0]))
        sys.stdout.flush()
        #Create temp dataframe for percent changes
        pct_change_df = row['data'][['buyingPrice', 'buyingCompleted', 'sellingPrice', 'sellingCompleted', 'overallPrice', 'overallCompleted']].copy()
        pct_change_df = pct_change_df.rename(index=int, 
                                         columns={'buyingPrice':'buyingPricePer', 'buyingCompleted':'buyingCompletedPer',
                                        'sellingPrice':'sellingPricePer', 'sellingCompleted':'sellingCompletedPer',
                                        'overallPrice':'overallPricePer', 'overallCompleted':'overallCompletedPer'})
        #Calculate percent change
        pct_change_df = pct_change_df.pct_change()
        
        #Join pct_change_df to existing row
        row['data'] = row['data'].join(pct_change_df)
        
    sys.stdout.write('\rDelta Creation Complete.')
    sys.stdout.flush()
        
    return item_records
        
def get_next_period_overall_price(item_records):
    """
        Append the next period's overall price to each entry
        
            @param item_records: the master item data structure
    """
    #Iterate through each item in item_records
    total_items = item_records.shape[0]
    for idx, row in item_records.iterrows():
        #Progress
        sys.stdout.write('\rProcessing Item: {0}/{1}'.format(idx, total_items))
        sys.stdout.flush()
        #Append new column for next period value
        new_col_name = 'overallPriceNext'
        row['data'][new_col_name] = row['data']['overallPricePer'].shift(-1)
        #Drop any rows that didn't have a next period price
        row['data'] = row['data'].dropna(subset=[new_col_name])
        
    sys.stdout.write('\rNext Period Price Calculation Complete.')
    sys.stdout.flush()
    
    return item_records
        
        
def generate_moving_averages(item_records, columns, window):
    """
        Calculate the moving average for each field with specified window size
        
            @param item_records: the master item data structure
            @param columns: a list of which columns (names) we are calculating average for
            @param window: the number of periods to average over -- e.g. 60 if doing 12 hour item records for 30 days
    """
    #Iterate through each item and calculate the moving average for each field
    for idx, row in item_records.iterrows():
        #Progress
        sys.stdout.write('\rProcessing Item: {0}/{1}'.format(idx, item_records.shape[0]))
        sys.stdout.flush()
        for col in columns:
            ma_col_name = col + 'MA'
            row['data'][ma_col_name] = row['data'][col].rolling(window=window).mean()
            
            #Drop all rows where MA is NaN
            row['data'] = row['data'].dropna(subset=[ma_col_name])
            
    sys.stdout.write('\rMoving Average Calculation Complete.')
    sys.stdout.flush()
            
    return item_records
            
def generate_z_scores(item_records, columns, outlier_thresh=3):
    """
        Generates z-scores for passed columns and appends to dataframe
        
            @param item_records: the master item data structure
            @param columns: the columns to calculate z-score of
            @param outlier_thresh: the zscore threshold after which we will remove a row
    """
    for idx, row in item_records.iterrows():
        #Iterate through each item and calculate z-scores for columns
        zscore_df = row['data'][columns].apply(zscore)
        #Rename columns
        rename_cols ={'buyingPricePer':'buyingPricePerZScore', 'buyingCompletedPer':'buyingCompletedPerZScore',
                                        'sellingPricePer':'sellingPricePerZScore', 'sellingCompletedPer':'sellingCompletedPerZScore',
                                        'overallPricePer':'overallPricePerZScore', 'overallCompletedPer':'overallCompletedPerZScore'}
        zscore_df.rename(columns=rename_cols, inplace=True)
        #Join zscore df to original item df
        row['data'] = row['data'].join(zscore_df)
        #Remove any outliers
        init_rows = row['data'].shape[0]
        for col in columns:
            col_name = col + 'ZScore'
            row['data'] = row['data'][abs(row['data'][col_name]) < outlier_thresh]
            
        #Display rows removed
        new_rows = row['data'].shape[0]
        print('Removed {0} Outlier Rows for {1}'.format(init_rows - new_rows, row['name']))
        
        #Reset Index
        row['data'].reset_index(drop=True, inplace=True)
        
    sys.stdout.write('\rZ-Score Calculation Complete.')
    sys.stdout.flush()
    
    return item_records
        
def remove_excess_columns(item_records):
    '''
        Removes columns that aren't necessary to train or evaluate the ANN
        
            @param item_records: the master item data structure
    '''
    cols_to_keep = ['buyingPricePer', 'sellingPricePer', 'overallPricePer',
                    'buyingCompletedPer', 'sellingCompletedPer', 'overallCompletedPer',
                    'overallPriceNext', 'tsFromCurrent', 'tsYtd',
                    'buyingPricePerMA', 'sellingPricePerMA', 'overallPricePerMA',
                    'buyingCompletedPerMA', 'sellingPricePerMA', 'overallPricePerMA']
    #Iterate through each item and drop excess cols
    for _, row in item_records.iterrows():
        row['data'] = row['data'][cols_to_keep]

def append_names_to_item_records(item_records):
    """
        Appends the item name to each row in item record data
        
            @param item_record: the master item data structure
    """
    for _, row in item_records.iterrows():
        row['data']['item_name'] = row['name']
    
def aggregate_cleaned_item_data(item_records):
    """
        Takes in cleaned master item data structure and returns a dataframe with all
        item data aggregated
        
            @param item_records: the master item data structure (cleaned)
            @returns: a new dataframe with aggregated item data
    """
    #Apply item names to all item records
    append_names_to_item_records(item_records)
    aggregate_df = pd.DataFrame()
    #Iterate through each item and append to aggregate df
    for ind, row in item_records.iterrows():
        aggregate_df = aggregate_df.append(row['data'], ignore_index=True)
    
    #Reset index in place
    aggregate_df.reset_index(drop=True, inplace=True)
    
    return aggregate_df

# Generate Item Records

In [2]:
#----- GENERATE ITEM RECORDS -----#
item_records = get_item_records_from_pickle('ge_data_1.pickle')

Processing Item: 506 / 507

# Initial Cleaning

In [3]:
item_records = clean_initial_data(item_records)
item_records = format_item_record_dates(item_records)

Data Cleaning Complete
Date Formatting Complete.

# Convert Data to Deltas

In [4]:
#----- CONVERT DATA TO DELTAS -----#
item_records = generate_train_and_test_deltas(item_records)

Delta Creation Complete.e bolts (e) -- ID: 92403939

# Get Period's Overall Price for Each Entry
These next period values are what we will be attempting to predict with the ANN. 

In [5]:
#----- GET NEXT PERIOD OVERALL PRICE FOR EACH ENTRY -----#
item_records = get_next_period_overall_price(item_records)

Next Period Price Calculation Complete.

# Generate 30-Day Averages For Columns

In [6]:
#------ CREATE 30 DAY AVERAGES FOR COLUMNS -----#
columns = ['overallPricePer', 'overallCompletedPer', 'buyingPricePer', 'buyingCompletedPer',\
           'sellingPricePer', 'sellingCompletedPer']
window = 1440 #24/.5 hour periods * 30 days = 1440 window
item_records = generate_moving_averages(item_records, columns, window)

Moving Average Calculation Complete.

# Generate Z-Scores and Remove Outliers

In [9]:
#----- Generate Z-Scores for Percentage columns and remove outliers -----#
item_records = generate_z_scores(item_records, columns)

Processing Item: 0/498Removed 1173 Outlier Rows for Cannonball
Processing Item: 1/498Removed 1450 Outlier Rows for Cannon base
Processing Item: 2/498Removed 1199 Outlier Rows for Cannon stand
Processing Item: 3/498Removed 1393 Outlier Rows for Cannon barrels
Processing Item: 4/498Removed 1692 Outlier Rows for Cannon furnace
Processing Item: 5/498Removed 441 Outlier Rows for Insect repellent
Processing Item: 6/498Removed 1074 Outlier Rows for Bucket of wax
Processing Item: 7/498Removed 1424 Outlier Rows for Candle
Processing Item: 8/498Removed 294 Outlier Rows for Bronze arrowtips
Processing Item: 9/498Removed 1051 Outlier Rows for Iron arrowtips
Processing Item: 10/498Removed 1088 Outlier Rows for Steel arrowtips
Processing Item: 11/498Removed 773 Outlier Rows for Mithril arrowtips
Processing Item: 12/498Removed 1486 Outlier Rows for Adamant arrowtips
Processing Item: 13/498Removed 804 Outlier Rows for Rune arrowtips
Processing Item: 14/498Removed 947 Outlier Rows for Opal bolt tips
Pr

c:\users\nate\appdata\local\programs\python\python37\lib\site-packages\scipy\stats\stats.py:2247: RuntimeWarning: Mean of empty slice.
  mns = a.mean(axis=axis)
c:\users\nate\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\nate\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
c:\users\nate\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
c:\users\nate\appdata\local\programs\python\python37\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: Other Series must have a name

# Remove Columns Not Required for ANN

In [ ]:
remove_excess_columns(item_records)
print(item_records.iloc[0]['data'])

# Aggregate all Cleaned Item Data

In [ ]:
aggregate_df = aggregate_cleaned_item_data(item_records)

In [13]:
item_records.iloc[34]['data']

,buyingCompleted,buyingPrice,overallCompleted,overallPrice,sellingCompleted,sellingPrice,ts,tsFromCurrent,date,tsYtd,...,sellingCompletedPer,overallPricePer,overallCompletedPer,overallPriceNext,overallPricePerMA,overallCompletedPerMA,buyingPricePerMA,buyingCompletedPerMA,sellingPricePerMA,sellingCompletedPerMA
8635,501.0,635.0,702,620,201.0,583.0,1.440925e+09,95961600.0,2015-08-30 09:00:00,20854800.0,...,-0.900888,0.035058,-0.654017,-0.059677,0.006049,18.817121,0.007825,38.686867,0.004306,30.915251
8636,1002.0,583.0,1002,583,NaN,NaN,1.440927e+09,95959800.0,2015-08-30 09:30:00,20856600.0,...,0.000000,-0.059677,0.427350,-0.022298,0.006012,18.817761,0.007768,38.687916,0.004304,30.915634
8637,620.0,570.0,620,570,NaN,NaN,1.440929e+09,95958000.0,2015-08-30 10:00:00,20858400.0,...,0.000000,-0.022298,-0.381238,0.022807,0.005969,18.816238,0.007728,38.686651,0.004155,30.906926
8638,697.0,583.0,711,583,14.0,614.0,1.440931e+09,95956200.0,2015-08-30 10:30:00,20860200.0,...,-0.930348,0.022807,0.146774,-0.005146,0.006068,18.815255,0.007704,38.687052,0.004295,30.897896
8639,630.0,583.0,658,580,28.0,527.0,1.440932e+09,95954400.0,2015-08-30 11:00:00,20862000.0,...,1.000000,-0.005146,-0.074543,0.003448,0.005910,18.815364,0.007693,38.684939,0.004007,30.899232
8640,986.0,582.0,1003,582,17.0,574.0,1.440934e+09,95952600.0,2015-08-30 11:30:00,20863800.0,...,-0.392857,0.003448,0.524316,-0.003436,0.005917,18.815992,0.007693,38.685706,0.004075,30.897948
8641,1031.0,581.0,2033,580,1002.0,579.0,1.440936e+09,95950800.0,2015-08-30 12:00:00,20865600.0,...,57.941176,-0.003436,1.026919,0.005172,0.005956,18.810441,0.007701,38.686055,0.004120,30.917812
8642,10.0,583.0,10,583,NaN,NaN,1.440938e+09,95949000.0,2015-08-30 12:30:00,20867400.0,...,0.000000,0.005172,-0.995081,-0.049743,0.005944,18.810422,0.007720,38.685919,0.004103,30.918490
8643,1671.0,555.0,1705,554,34.0,534.0,1.440940e+09,95947200.0,2015-08-30 13:00:00,20869200.0,...,-0.966068,-0.049743,169.500000,0.028881,0.005907,18.927264,0.007687,38.797869,0.004045,30.917779
8644,2986.0,570.0,3056,570,70.0,577.0,1.440941e+09,95945400.0,2015-08-30 13:30:00,20871000.0,...,1.058824,0.028881,0.792375,-0.042105,0.005954,18.923611,0.007706,38.796402,0.004143,30.910322


In [7]:
item_records

,id,name,data
0,2,Cannonball,buyingCompleted buyingPrice overallCo...
1,6,Cannon base,buyingCompleted buyingPrice overallCo...
2,8,Cannon stand,buyingCompleted buyingPrice overallCo...
3,10,Cannon barrels,buyingCompleted buyingPrice overallCo...
4,12,Cannon furnace,buyingCompleted buyingPrice overallCo...
5,28,Insect repellent,buyingCompleted buyingPrice overallCo...
6,30,Bucket of wax,buyingCompleted buyingPrice overallCo...
7,36,Candle,buyingCompleted buyingPrice overallCo...
8,39,Bronze arrowtips,buyingCompleted buyingPrice overallCo...
9,40,Iron arrowtips,buyingCompleted buyingPrice overallCo...
